In [1]:
#Run this if you are running the program for the first time
!pip install nomic
!pip install -U langchain-nomic langchain_community tiktoken langchain-openai chromadb langchain

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 164.7 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 373.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 255.6 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 1.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for nomic: filename=nomic-3.0.27-py3-none-any.whl size=44143 sha256=e849ecd168dbc424a493d289d5b2bac7e3ce93d06a6633f9124d7875a4370885
  Stored in directory: /Commjhub/jupyterhub/home/seanmckeown1/.cache/

In [2]:
from langchain_nomic.embeddings import NomicEmbeddings
import os

In [ ]:
# Convert data into text functions

import os

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_nomic import NomicEmbeddings
from langchain_nomic.embeddings import NomicEmbeddings

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

pdf_list = ["Style.pdf", "DEI.pdf", "34th.pdf", "sports.pdf"]
urls = [
'https://yoast.com/slug/', 'https://www.semrush.com/blog/what-is-a-url-slug/?kw=&cmp=US_SRCH_DSA_Blog_EN&label=dsa_pagefeed&Network=g&Device=c&kwid=dsa-2185834088336&cmpid=18348486859&agpid=156019556762&BU=Core&extid=97592280163&adpos=', 'https://www.upwork.com/resources/how-to-write-seo-content','https://authorservices.wiley.com/author-resources/Journal-Authors/Prepare/writing-for-seo.html','https://www.semrush.com/blog/seo-writing/','https://www.semrush.com/kb/839-how-to-write-seo-articles-four-steps','https://www.flowmatters.com/blog/a-practical-guide-on-how-to-write-seo-articles/','https://www.maropost.com/how-to-combine-seo-and-email-marketing-for-better-rankings/','https://www.webfx.com/seo/learn/email-marketing-tips-to-improve-seo/','https://sendgrid.com/en-us/blog/seo-and-email-marketing','https://www.emailonacid.com/blog/article/email-marketing/seo-connections/','https://coalitiontechnologies.com/blog/strategic-seo-tips-for-email-marketing','https://optinmonster.com/101-email-subject-lines-your-subscribers-cant-resist/','https://www.wordstream.com/blog/ws/2014/03/31/email-subject-lines','https://www.constantcontact.com/blog/good-email-subject-lines/','https://blog.hubspot.com/marketing/best-email-subject-lines-list'
]

def load_csv(csv):
    loader = CSVLoader(file_path= csv)
    data = loader.load()
    return data

def load_url(url_list):
    urls = url_list
    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]
    return docs_list

def load_pdf(pdf_list):
    pdfs = pdf_list
    output = [PyPDFLoader(pdf).load() for pdf in pdfs]
    pdfs_list = [item for sublist in output for item in sublist]
    return pdfs_list

data = load_csv("merged_stats.csv")
docs_list = load_url(urls)
pdfs_list = load_pdf(pdf_list)

#Splitting
def splitter(data, docs_list, pdfs_list):
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=7500, chunk_overlap=100
    )
    
    doc_splits = text_splitter.split_documents(data)
    
    url_text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=7500, chunk_overlap=100
    )
    
    url_splits = url_text_splitter.split_documents(docs_list)
    
    pdf_text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=7500, chunk_overlap=100
    )
    
    pdf_splits = pdf_text_splitter.split_documents(pdfs_list)

    return doc_splits, url_splits, pdf_splits

doc_splits, url_splits, pdf_splits = splitter(data, docs_list, pdfs_list) 

# Vector DB for Articles.csv
csv_vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
csv_retriever = csv_vectorstore.as_retriever()

# Vector DB for SEO 

url_vectorstore = Chroma.from_documents(
    documents=url_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
url_retriever = url_vectorstore.as_retriever()

# Vector DB for Writing Style Documents 

pdf_vectorstore = Chroma.from_documents(
    documents=pdf_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
pdf_retriever = pdf_vectorstore.as_retriever()

template = """ Pretend you are an SEO expert for a company called "The Daily Pennsylvanian" that does journalism for the University of Pennsylvania.  
Answer the question based only on the following context: 

Information about previous articles as well as their performance metrics can be found through: {context}

Information about SEO Optimization can be found through: {context1}

The Daily Pennsylvanian writing style guide and tips can be found through: {context2}

Question: Output 3 potential URL Slugs and SEO titles based on the provided Drafted Title and Content 
make sure that the URL Slug is in the correct format that a URL Slug should be and that 
the SEO title is search engine optimized and concise. DO NOT ASSUME ANY INFORMATION, make the title based ONLY on the information told in the question {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3"
model_local = ChatOllama(model=ollama_llm)

# Chain
# take the question, chroma search, gives back chunks, that 
# context , 1 , 2 , 3 seperate objects retrievers
chain = (
    {"context": csv_retriever, "context1" : url_retriever, "context2" : pdf_retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [54]:
!pip install -U langchain-anthropic

2253.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [60]:
# Convert data into text functions
import anthropic
from langchain_anthropic import ChatAnthropic
import os

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_nomic import NomicEmbeddings
from langchain_nomic.embeddings import NomicEmbeddings

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

pdf_list = ["Style.pdf", "DEI.pdf", "34th.pdf", "sports.pdf"]
urls = [
'https://yoast.com/slug/', 'https://www.semrush.com/blog/what-is-a-url-slug/?kw=&cmp=US_SRCH_DSA_Blog_EN&label=dsa_pagefeed&Network=g&Device=c&kwid=dsa-2185834088336&cmpid=18348486859&agpid=156019556762&BU=Core&extid=97592280163&adpos=', 'https://www.upwork.com/resources/how-to-write-seo-content','https://authorservices.wiley.com/author-resources/Journal-Authors/Prepare/writing-for-seo.html','https://www.semrush.com/blog/seo-writing/','https://www.semrush.com/kb/839-how-to-write-seo-articles-four-steps','https://www.flowmatters.com/blog/a-practical-guide-on-how-to-write-seo-articles/','https://www.maropost.com/how-to-combine-seo-and-email-marketing-for-better-rankings/','https://www.webfx.com/seo/learn/email-marketing-tips-to-improve-seo/','https://sendgrid.com/en-us/blog/seo-and-email-marketing','https://www.emailonacid.com/blog/article/email-marketing/seo-connections/','https://coalitiontechnologies.com/blog/strategic-seo-tips-for-email-marketing','https://optinmonster.com/101-email-subject-lines-your-subscribers-cant-resist/','https://www.wordstream.com/blog/ws/2014/03/31/email-subject-lines','https://www.constantcontact.com/blog/good-email-subject-lines/','https://blog.hubspot.com/marketing/best-email-subject-lines-list'
]

def load_csv(csv):
    loader = CSVLoader(file_path= csv)
    data = loader.load()
    return data

def load_url(url_list):
    urls = url_list
    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]
    return docs_list

def load_pdf(pdf_list):
    pdfs = pdf_list
    output = [PyPDFLoader(pdf).load() for pdf in pdfs]
    pdfs_list = [item for sublist in output for item in sublist]
    return pdfs_list

data = load_csv("merged_stats.csv")
docs_list = load_url(urls)
pdfs_list = load_pdf(pdf_list)

#Splitting
def splitter(data, docs_list, pdfs_list):
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=7500, chunk_overlap=100
    )
    
    doc_splits = text_splitter.split_documents(data)
    
    url_text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=7500, chunk_overlap=100
    )
    
    url_splits = url_text_splitter.split_documents(docs_list)
    
    pdf_text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=7500, chunk_overlap=100
    )
    
    pdf_splits = pdf_text_splitter.split_documents(pdfs_list)

    return doc_splits, url_splits, pdf_splits

doc_splits, url_splits, pdf_splits = splitter(data, docs_list, pdfs_list) 

# Vector DB for Articles.csv
csv_vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
csv_retriever = csv_vectorstore.as_retriever()

# Vector DB for SEO 

url_vectorstore = Chroma.from_documents(
    documents=url_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
url_retriever = url_vectorstore.as_retriever()

# Vector DB for Writing Style Documents 

pdf_vectorstore = Chroma.from_documents(
    documents=pdf_splits,
    collection_name="rag-chroma",
    embedding=NomicEmbeddings(model="nomic-embed-text-v1"),
)
pdf_retriever = pdf_vectorstore.as_retriever()

template = """ Pretend you are an SEO expert for a company called "The Daily Pennsylvanian" that does journalism for the University of Pennsylvania.  
Answer the question based only on the following context: 

Information about previous articles as well as their performance metrics can be found through: {context}

Information about SEO Optimization can be found through: {context1}

The Daily Pennsylvanian writing style guide and tips can be found through: {context2}

Question: Output 3 potential URL Slugs and SEO titles based on the provided Drafted Title and Content 
make sure that the URL Slug is in the correct format that a URL Slug should be and that 
the SEO title is search engine optimized and concise. DO NOT ASSUME ANY INFORMATION, make the title based ONLY on the information told in the question {question}
"""
prompt = ChatPromptTemplate.from_template(template)
 
# Local LLM
llm_name = "claude-3-opus"
model_remote = ChatAnthropic(model="claude-3-opus")  

# Chain
# take the question, chroma search, gives back chunks, that 
# context , 1 , 2 , 3 seperate objects retrievers
chain = (
    {"context": csv_retriever, "context1" : url_retriever, "context2" : pdf_retriever, "question": RunnablePassthrough()}
    | prompt
    | model_remote
    | StrOutputParser()
)

In [12]:
def activate_chain(t, c):
    output = chain.invoke(f'Title: {t} and Content: {c}')
    return output
    
title = (str(input("Please give me your suggested title, I will optimize it! ")))
print()
content = str(input("Please tell me what you are writing about: "))

print("response")
print(activate_chain(title, content))

Please give me your suggested title, I will optimize it!  


Please tell me what you are writing about:  They say that when you get worked up about something to count down from ten or repeat a mantra to try to calm down. But what happens when your chest gets so tight that you can’t think about what comes after seven? What about when you stomach is lurching up your throat at what feels like 1,000 miles an hour? How are you supposed to remember a mantra when you are trying to stop your eyes from darting from right to left, constantly in search of something but you’re not sure what, all while you are trying to figure out how to escape your shirt that is inexplicably and suddenly trying to pull you inside out?  I had my first panic attack when I was 17. I was sitting in my AP Stat class, third row back, fourth seat from the right. It was a crisp early October day and I remember the vivid fall colors adorning the trees outside the windows to my right: burnt orange leaves with the veins dyed deeper orange hues, blood red, perfectly symmetrical foliage

response
Based on the provided content, here are three potential URL Slugs and SEO titles:

**Option 1:**
URL Slug: penn-williams-hall-uncomfortable-classroom-temperatures
SEO Title: "Uncomfortable Classroom Temperatures: Williams Hall Students Face Heat Wave"

**Option 2:**
URL Slug: overheated-classrooms-disrupt-penn-learning
SEO Title: "Overheated Classrooms Disrupt Learning at Penn: Williams Hall's Temperature Issues Cause Concern"

**Option 3:**
URL Slug: penn-williams-hall-classroom-temperature-issues
SEO Title: "Temperature Issues in Williams Hall Classrooms Cause Concern for Students and Faculty at Penn"

These options aim to capture the essence of the article, highlighting the uncomfortable temperatures in Williams Hall classrooms and their impact on students' learning experiences. The URL Slugs follow the standard format of including relevant keywords and being concise.


In [42]:
import openai
import gradio as gr
from dotenv import load_dotenv
import os
import requests
import json

# Load the environment variables and setup (same as before)
load_dotenv()
model = os.getenv("MODEL_NAME", "llama3:latest")

# Define the RAG chain setup here (as per your previous code)
# Assuming all imports and setup from your RAG code are done correctly

In [ ]:

def chat(input_text, dept, title, content, chat_history):
    chat_history = chat_history or []
    global context
    
    # Assemble the prompt text if necessary
    prompt_text = f"Given that I work for this department {dept} and have the article title of this: {title}, here is what the article is about {content} answer this question: {input_text}"
    
    chat_history.append((input_text, chain.invoke(prompt_text)))
    
    # Clear input fields and maintain the chat history
    return chat_history, chat_history, "", "", "", ""

# Adjust the chain setup
chain = (
    {"context": csv_retriever, "context1": url_retriever, "context2": pdf_retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

# Setup Gradio UI
theme = gr.themes.Base(
    primary_hue="red",
    secondary_hue="red",
    neutral_hue="slate",
)

with gr.Blocks(theme=theme) as demo:
    gr.Markdown("<h1><center>Daily Pennsylvanian SEO Optimizer</center></h1>")
    chatbot = gr.Chatbot()
    title = gr.Textbox(placeholder="Title here", label="Article Title")
    content = gr.Textbox(placeholder="Article content here", label="Article Content")
    input_box = gr.Textbox(placeholder="Chat with the GPT", label="Question")
    dept = gr.Dropdown(["Under the Button", "34th Street", "Quaker Nation", "DP General"], label="Department", info="Please tell me what department you are writing for!", allow_custom_value = True)
    state = gr.State()

    submit = gr.Button("SEND")
    clear = gr.Button("CLEAR")
    reset_chat = gr.Button("RESET CHAT HISTORY")

    submit.click(chat, inputs=[input_box, dept, title, content, state], outputs=[chatbot, state, input_box, dept, title, content])
    clear.click(lambda: ([], None, None, None, [], []), inputs=None, outputs=[chatbot, input_box, dept, title, content, state], queue=False)
    reset_chat.click(lambda: ([]), inputs=None, outputs=[chatbot], queue=False)

demo.launch(debug=True, share=True)

IMPORTANT: You are using gradio version 4.16.0, however version 4.29.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://eb966e7766f9167813.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/anaconda3/share/jupyter/venv/py3-11_comm4190/lib/python3.11/site-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/share/jupyter/venv/py3-11_comm4190/lib/python3.11/site-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/share/jupyter/venv/py3-11_comm4190/lib/python3.11/site-packages/gradio/blocks.py", line 1561, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/share/jupyter/venv/py3-11_comm4190/lib/python3.11/site-packages/gradio/blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anacon